# __Perform Topology Optimization with TOVAE__
#### <u>Overview</u>: This notebook performs inverse design using the trained TO-VAE. The Material Property Predictor(s) perform inference on the Latent Vector (without the volume fraction component). The prediction can be fed to a customizable loss function that implements an optimization problem. The loss function contains the objective function and the constraints, but the values are produced by the MPPs performing inference on the Latent Vector. Therefore, performing gradient descent optimization on the gradient of the latent vector with respect to the loss function reults in a latent vector that the MPPs calculate to have the targeted material properties. The final latent vector is then decoded into a topology using the decoder portion of the TOVAE, and notionally the topology exhibits the targeted material properties. Validation must be performed using FEA homogenization.

__Optimization production loop__:

1. Randomly select a topology from the dataset
2. Generate its latent vector using the TO-VAE Encoder
3. Produce an initial predicted MPV using the MPP portion of the TO-VAE
4. Calculate loss
5. Calculate gradient of loss with respect to the Latent Vector
6. Adjust the LV using a gradient-descent optimizer
7. Repeat 4-6 for set number of iterations
8. Upon completion, decode the final latent vector into an array
9. Apply FEA to determine material properties
10. Repeat 1-9 for desired number of runs

![TO-VAE diagram](./media/to_vae_w_512.png)

In [1]:
# For setting the directory references for the entire package
from ML_workflow_utils_v3.PackageDirectories import PackageDirectories as PD   

# This code automatically sets the rootpath as the directory the entire package is contained in, which is then called to initialize the PackageDirectories class below
import os
# check current path if desired
# currentpath = os.getcwd()
# print(currentpath)

os.chdir('../../../')
rootpath = os.getcwd()
# print(rootpath)

# Alternately, rootpath can be set manually
# rootpath = 'filepath/containing/entire/ML_package/'

directory = PD(rootpath = rootpath)

In [2]:
import pandas as pd
import numpy as np
import json
import glob
import os
import time

from ML_workflow_utils_v3.Dataset_Preprocessor import Dataset_Preprocessor as DataP
source_data_path = directory.source_data_path
voxel_dir = directory.voxeltopo_path


import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F


import torch.utils.data as data
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


from ML_workflow_utils_v3.Voxel_Mesh_Utils import target_binarray_threshold, Plot_Array 
from ML_workflow_utils_v3.Model_Weights_Util import convert_state_dict
from ML_workflow_utils_v3.Misc_Utils import scale, unscale, save_dict_to_pickle, load_dict_from_pickle
from collections import OrderedDict
from scipy.io import savemat


In [3]:
# Set notebook path (nbpath) for the notebook - all future directory/filepath calls
nbpath = directory.nb_3_2_path

# Set paths for database CSVs and filepath of voxel meshes
source_data_path = directory.source_data_path
voxel_dir = directory.voxeltopo_path

In [4]:
#### Material Properties #####
matprops = ['volFrac', 
         'CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',
         'CH_12 scaled', 'CH_13 scaled','CH_23 scaled',
         'EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',
         'GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled', 
         'vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',
         'KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled', 
         'kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']

matprops_by_module = [['volFrac',], 
                      ['CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',],
                      ['CH_12 scaled', 'CH_13 scaled','CH_23 scaled',],
                      ['EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',],
                      ['GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled',],
                      ['vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',],
                      ['KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled',],
                      ['kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']]

num_props = len(matprops) # for determining the output dimension

In [5]:
def get_latent_vec(rvae, arraypath):
    """
    Produces latent vector, given an input array in .npz file format (numpy compressed format)

    Args:
        rvae: loaded instance of trained TO-VAE
        arraypath (str): path to input voxel mesh array
    """
    arr = np.load(arraypath)['arr_0']
    if arr.dtype != 'int8':
        arr = arr.astype(np.int8)
    else:
        pass
    arr = np.expand_dims(arr, (0,1))    
    arr = torch.Tensor(arr).to(dtype=torch.float32).to(device)
    latent_vec = rvae(arr)[1]
    
    return(latent_vec)

In [6]:
from ML_workflow_utils_v3.TO_VAE import TOVAE

In [7]:
"""
Load the multiphysics database of homogenized properties
"""

# Name of the CSV file that contains the homogenized material properties - "statdb" = static database
statdb_csvname =  'topology_multiphysics_database_by_partno.csv' #'full_db_cao_may24_PNs_final_plttube_reduced_scaled.csv'

# database (db) path
sdbpath = os.path.join(source_data_path, statdb_csvname)

sdb = pd.read_csv(sdbpath)


dyndb_csvname = 'dynamic_static_database_scaled.csv'

dyndbpath = os.path.join(source_data_path, dyndb_csvname)

dyndb = pd.read_csv(dyndbpath)


In [8]:
# The values for m and b were determined using the dyndb, predicting CH_11 from the respective material properties (scaled) below
dyn_linreg_dict = {'plateau stress' : {'m': 0.3887291380713647, 'b': 0.010396286544529405},
                   'energy absorbed': {'m': 0.42632711358611763, 'b': 0.018541292815752786}}

In [9]:
def calculate_dynamic_property(property='plateau stress', C11=1.0, dyn_linreg_dict = dyn_linreg_dict):
    """
    Calculate predicted dynamic property based on input C[H]_11 value

    property (str): set to "plateau stress" or "energy absorbed" - will call the applicable linear regression coefficients from the dictioary
    C11 (float): value of C11
    dyn_linreg_dict (dict): dictionary containing the coefficients for the linear regressions that predict C11 from plateau stress and energy absorbed, respectively

    Returns:
        tuple: tuple of the symbol/variable of the output property, either sig_pl for plateau stress or W for energy absorbed, along with the value
    """
    if property == 'plateau stress':
        prop_out = 'sig_pl'
    elif property == 'energy absorbed':
        prop_out = 'W'
    else:
        Exception("property must be \'plateau stress\' or \'energy absorbed\'")
    
    
    m = dyn_linreg_dict[property]['m']
    b = dyn_linreg_dict[property]['b']

    dynprop_calc = (C11 - b) / m

    return prop_out, dynprop_calc

    

In [10]:
# Defining variables for VAE configuration

# latent_dim is the dimension of the "encoded"
latent_dim = 512

In [11]:
from ML_workflow_utils_v3.TO_VAE import TOVAE

In [12]:
# If multiple GPUs are available, set to True
gpu_parallel = False

if gpu_parallel:
    tovae = torch.nn.DataParallel(TOVAE(latent_dim, matprops, matprops_by_module).to(device))
else:
    tovae = TOVAE(latent_dim, matprops, matprops_by_module).to(device)


In [ ]:

"""
If loading the TopoGen-VAE that was shipped with this package, execute these three lines of code
"""
cp_dir = directory.topopt_vae_path
cp_name = "TO_VAE_pretrained_model_weights.pth"
cp_path = os.path.join(cp_dir, cp_name)


"""
If loading a model trained in Notebook 3.1 (3_1_Train_TO_VAE), comment out the three lines of code
above by highlighting and pressing Ctrl+/, or putting '#' at the beginning of each line

"""

# cp_dir = os.path.join(directory.nb_3_1_path, 'model_CPs')
# cp_name = 'TO_VAE_allprops_28AUG24' # placeholder, replace
# cp_path = os.path.join(cp_dir, cp_name)

In [ ]:
"""
This command loads the trained model's weights

The included pre-trained model weights were produced using a multi-GPU training setup, therefore the key of each layer contains "module.", 
which is how torch.nn.DataParallel() creates state dictionaries. The call of "convert_state_dict"
If using more than one GPU for topology production, set "convert_weight_keys_to" to 'parallel'
"""

convert_weight_keys_to = 'non-parallel'

model_weights = convert_state_dict(cp_path, convert_to = convert_weight_keys_to)


tovae.load_state_dict(model_weights)

In [ ]:
# Set TOVAE to evaluation mode
tovae.eval()

In [ ]:
# Set a variable to call the decoder module of the TopoGen-VAE and set it to evaluation mode
if convert_weight_keys_to == 'non-parallel':
    decoder = tovae.decoder
elif convert_weight_keys_to == 'parallel':
    decoder = tovae.module.decoder

decoder.to(device)

## __Topology Optimization with Optimization Loss Function__

#### The Optimization Loss Function (OLF) is an extremely flexible structure for representing an optimization problem. It can contain the objective function and the constraints. As the pre-trained TOVAE predicts all material properties, optimization can be performed to target or constrain as many material properties as desired.

#### The loss function is a sum of terms, where each term corresponds to an objective function or a constraint.

#### <u>Objective Function</u>: Can have two forms (using $C^{H}_{11}$ as the example target property):
1. Maximize $C_{11}^H$ to achieve the highest \sigma_{pl} possible 
- <u>Objective function</u>: &emsp; &emsp; $max\ C_{11}^H$
- Alternatively,  &emsp; &emsp; &emsp; &nbsp; &nbsp; $min\ (-C_{11}^H)$


2. Target $C_{11}^H$ to a specific value
- <u>Objective function</u>:&nbsp;&nbsp;$min\left|C_{11,topo}^H-\widehat{C_{11}^H}\right|$

- <u>Note</u>: There can be as many targeted properties as desired, forming the Targeted Material Property Vector, or $\widehat{MPV}$.

The loss function for MPV made of $n$ material properties $p_{n}$

$min     Loss_{opt} =$

$ min       \left[ \left|{p}_{1}-\widehat{{p}_{1}}\right| + \left|{p}_{2}-\widehat{{p}_{2}}\right| + ... + \left|{p}_{n}-\widehat{{p}_{n}}\right| \right]$


#### <u>Constraints</u>: are calculations comparing the Latent Vector's material properties predicted by the MPP network, gated with the Rectified Linear Unit (ReLU) function. The ReLU function is as follows:

$\text{ReLU}(x) =
\begin{cases} 
x, & \text{if } x \geq 0 \\
0, & \text{otherwise}
\end{cases}$

or 

ReLU(x) = max(0, x)

Applying to a constraint such as

$ x \leq C$

rearranged as

$ x - C \leq 0$

The gradient descent loss function - the OLF - penalizes positive values as it drives the loss function value to a minimum. Therefore, the ReLU function allows for only contributing an error if the constraint is greater than zero. For the constraint above there are two cases:

(i) $x < C$: The constraint is not violated and should not contribute to the Loss function, therefore it should be zeroed.
(ii) $x > C$: The constraint is violated and should contribute a positive number to the loss function, which Implemented with the ReLU function, the constraint $Z$ is:

$Z = ReLU(x - C)$

Thus in case (i), Z = 0, and in case (ii), Z > 0 and contributes to the loss function, which the gradient descent optimizer drives to zero. 

For a "greater than" constraint, $ x \geq C$, or $ x - C \geq 0$, we mlutiply the term by -1 to get $ C - x \leq 0$, so the constraint $Z$ is

$Z = ReLU(C - x)$

In the loss function provided below, the targeted properties are $C^{H}_{11}$, volume fraction ("volFrac" in the database), and the axial fluid permeabilities $K^{H}_{11}, K^{H}_{22}, K^{H}_{33}$. The objective function is to maximize $C^{H}_{11}$ while setting a lower bound on all three permeability values. The topology is optimized to have a targeted volume fraction.

<u>Note</u>: the MPP can produce predicted material properties that are outside the values of the actual data - in the case of scaled data, outside of the range of [0,1]. We implement bounds such that the material properties cannot exceed 1.0. We implement the "big M" method to 

## Setting up for the optimization runs

### If targeting a specific MPV, set `optimiz_loss` to  `False` and define `tgt_mpv` with the properties you want, in the order you want

In [17]:
# Set number of iterations to drive OLF as low as achievable - adjust as preferred, trade-off is duration of each run
num_iterations = 25000
# Number of runs to perform
num_runs = 100

# If maximizing a value, not just targeting a MPV, set to True
optimiz_loss = True

# set target volume fraction and configure loss function
tgt_vf = 0.2

# Convert to PyTorch tensor
# tgt_vf_torch = torch.tensor(tgt_vf).unsqueeze(0).unsqueeze(0).cuda()
tgt_vf_torch = torch.tensor(tgt_vf).unsqueeze(0).unsqueeze(0).to(device)



In [18]:
# Set and create the output folder
date = '23SEP24'
batch_folder_name = f"batch0_{date}"
batch_path = os.path.join(nbpath, 'TO_production_runs', batch_folder_name)
os.makedirs(batch_path, exist_ok=True)

matdir = os.path.join(batch_path, 'mat_files')
os.makedirs(matdir, exist_ok=True)
plotdir = os.path.join(batch_path, 'array_plots')
os.makedirs(plotdir, exist_ok=True)


In [19]:
class Objective_Function(nn.Module):
    def __init__(self, matprop, config_dict):
        
        super(Objective_Function, self).__init__()

        # matprop, type='max', weight=1.0
        

        self.matprop = matprop
        self.of_type = config_dict['type']
        self.weight =  config_dict['weight']

        if self.of_type != 'min' and self.of_type != 'max':
            raise Exception("of_type must be \'max\' or \'min\'")
       

    def calculate_objfn_value(self, x):

        if self.of_type == 'max':
            self.objfn_value = -1 * self.weight * x
        elif self.of_type == 'min':
            self.objfn_value = self.weight * x
        # print(self.objfn_value)
        # self.objfn_value = torch.Tensor(np.asarray(self.objfn_value)).unsqueeze(0).requires_grad(True)
        return self.objfn_value

In [20]:
class MatProp_Constraints(nn.Module):
    def __init__(self, matprop, config_dict: dict):
        
        super(MatProp_Constraints, self).__init__()

        self.matprop = matprop

        self.upper = config_dict['upper']
        self.lower = config_dict['lower']
        self.bigM =  config_dict['bigM']
        
        if 'upper' in self.bigM:
            self.W_up =  config_dict['bigM_val']
        else:
            self.W_up =  config_dict['up_W']
        if 'lower' in self.bigM:
            self.W_low = config_dict['bigM_val']
        else:
            self.W_low = config_dict['low_W']
            

        self.upval = 1.0
        self.lowval = 0.0

    def calculate_loss_terms(self, x):

        total_loss = torch.Tensor([0.0]).to(device).requires_grad_(True)
        
        if self.upper is not None:      
            self.upper_bound_loss = torch.relu(x - self.upval) * self.W_up
            total_loss = total_loss + self.upper_bound_loss
        
        if self.lower is not None:
            self.lower_bound_loss = torch.relu(self.lowval - x) * self.W_low
            total_loss + total_loss + self.lower_bound_loss

        return total_loss


### Set the configuration for the optimization problem.
#### The `config_dict` allows for setting the objective function/variables, which are targeted as maximize or minimize, and the constraints. It takes the following keywords:

#### `objective variables`: entry is a nested `dict`:
##### <u>first level key(s)</u>: name(s) of material properties to target with objective function, e.g., `'CH_11 scaled'`. Keys in next level `dict` as follows:
## <u>__Note</u>:__ For predicting dynamic material properties - plateau stress $\bold{\sigma_{pl}}$ and energy absorbed $\bold{W}$, one objective variable must be $\bold{C^{H}_{11}}$
##### `'type'`: `'max'` or `'min'` 
##### `'weight'` - default is `1.0`, set to desired weight to contribute to loss function. This value slightly weights the constraint in the gradient descent process, allowing emphasis on objective variable(s)/function(s)

#### `constraints`: entry is a nested `dict`:
##### <u>first level key(s)</u>: name(s) of material properties to target with objective function, e.g., `'KH_11 scaled'`. Keys in next level `dict` as follows:
##### `'upper'`: `True` or `False` - if True, implement upper bound
##### `'lower'`: `True` or `False` - if True, implement lower bound
##### `'bigM'`: `list` contains `'upper'`, `'lower'`, both, or empty - implements "big-M" penalty on specified constraint(s) to ensure it is not violated
##### `'upval'`: value of upper-bound constraint. Defaults to 1.0. This ensures that any constraint will not exceed 1.0, the maximum [scaled] value for the material property. Applicable if the target constraint on the material property is a "greater than or equal to" constraint. If a "less than or equal to" constraint, this value may be set lower to the desired upper bound.
##### `'lowval'`: value of lower-bound constraint. Defaults to 0.0.
##### `'bigM_val'`: sets value of big-M penalty. Defaults to 10,000 (1e5). Likely not necessary to change this value.
##### `'up_W'`: value of weight on upper-bound constraint. Defaults to 1.0. If not using big-M penalty, this value slightly weights the constraint in the gradient descent process, allowing emphasis of certain constraints
##### `'low_W'`: value of lower-bound constraint. Defaults to 1.0. If not using big-M penalty, this value slightly weights the constraint in the gradient descent process, allowing emphasis of certain constraints


In [21]:
# it is recommended to always include volume fraction as a constraint

config_dict = {'objective variables': {'CH_11 scaled': {'type': 'max', 'weight': 1.0}},
'constraints': {'volFrac': {'upper': True, 'lower': True, 'bigM': [], 
                                          'upval':0.4, 'lowval': 0.0, 'bigM_val': 1e5, 'up_W': 1.0, 'low_W': 1.0},
                'KH_11 scaled': {'upper': True, 'lower': False, 'bigM': ['upper'], 
                                          'upval':1.0, 'lowval': 0.0, 'bigM_val': 1e5, 'up_W': 1.0, 'low_W': 1.0},
                'KH_22 scaled': {'upper': True, 'lower': False, 'bigM': ['upper'], 
                                          'upval':1.0, 'lowval': 0.0, 'bigM_val': 1e5, 'up_W': 1.0, 'low_W': 1.0},
                'KH_33 scaled': {'upper': True, 'lower': False, 'bigM': ['upper'], 
                                          'upval':1.0, 'lowval': 0.0, 'bigM_val': 1e5, 'up_W': 1.0, 'low_W': 1.0}
}}


"""
If targeting a specific material property vector using L1 Loss, use this format:

dictionary keys are the material properties, their values are the decimal values of the target MPV
example below:
"""
# config_dict = {'CH_11 scaled': 0.5, 'volfrac': 0.3, 'KH_11 scaled': 0.15, 'KH_22 scaled': 0.15, 'KH_33 scaled': 0.15}
## set the target MPV in a Tensor as well
# tgt_mpv = np.asarray([0.5, 0.3, 0.15, 0.15, 0.15])
# tgt_mpv = torch.Tensor(tgt_mpv).unsqueeze(0).to(device)

# set to True if CH_11 is an objective variable for predicting dynamic material properties
dynamic_topopt = True

# set a suffix for naming the produced arrays that reflects the material properties targeted
run_suffix = 'C1VFK123'


In [22]:
# Create a dataframe for holding predicted material properties of latent vector at the end of training

cols = []
for key in config_dict.keys():
    for matprop in config_dict[key].keys():
        if matprop == 'volFrac':
            cols.append(matprop)
        else:
            matprop = matprop.split(' ')[0]
            cols.append(matprop)
            unscaled = f'{matprop} unscaled'
            cols.append(unscaled)
        

predsdf =  pd.DataFrame(columns = cols)

In [23]:
class GradOpt_MAE_Loss_module(nn.Module):

    def __init__(self, tovae, olf_config: dict, matprop_mpp_dict: dict,):
        """
        Inputs for the gradient descent setup module

        Args:
            tovae: loaded instance of TO-VAE
            olf_config (dict): configuration dictionary for objective loss function
            matprop_mpp_dict (dict): dictionary that looks up the MPP that predicts each material property
        """

        super(GradOpt_MAE_Loss_module, self).__init__()

        self.olf_config = olf_config

        self.matprops = []
        self.mpv_dict = {}
        self.tgt_mpv = []

        idx = 0
        for key in olf_config.keys():
            self.matprops.append(key)
            self.mpv_dict[key] = idx
            self.tgt_mpv.append(olf_config[key])
            idx +=1

        self.tgt_mpv = torch.Tensor(np.asarray(self.tgt_mpv)).unsqueeze(0).to(device)

        
        # loop through the configuration dictionary
        for key in olf_config.keys():
            sub = olf_config[key]
            for skey in sub.keys():
                # add to list of the material properties
                self.matprops.append(skey)

                #
                self.mpv_dict[skey] = idx
                idx += 1


        self.mpp_dict = matprop_mpp_dict 

        self.mpps = nn.ModuleList()

        self.mpp_idx = []

        
        for mp in self.matprops:
            mpp_id = self.mpp_dict[mp][0] # pulls the name of the MPP based on the 
            mpp = getattr(tovae, mpp_id) # pulls the MPP module from the TOVAE
            self.mpps.append(mpp)        # adds to ModuleList
            self.mpp_idx.append(self.mpp_dict[mp][1]) # takes the index of the material property in the output of the MPP, e.g., if the MPP predicts KH_11, KH_22, and KH_33, then KH_22 is at index [1]

    def get_mpv_pred(self, latent_vec):
        """
        Produces material property prediction vector of selected material properties, based on the property name
        and its index in the output from its MPP. E.g., if the MPP predicts KH_11, KH_22, and KH_33, then KH_22 is at index [1]

        Args:
            latent_vec (torch.Tensor): latent vector as input

        Returns:
            torch.Tensor: tensor of the predicted material properties
        """

        preds = []
        for mpp, idx in zip(self.mpps, self.mpp_idx):
            pred = mpp(latent_vec.to(device))
            pred = list(pred.detach().cpu().numpy()[0,:])[idx]

            preds.append(pred)

        preds = torch.from_numpy(np.asarray(preds)).unsqueeze(0).to(device)

        return preds



    def forward(self, latent_vec):
        """
        Calls the objective function and constraints, adds them together, returns the total loss value

        Args:
            latent_vec (torch.Tensor): latent vector

        Returns:
            torch.Tensor: predicted MPV
        """

        self.mpv_pred = self.get_mpv_pred(latent_vec)

        return self.mpv_pred

In [24]:
class GradOpt_Optimization_Loss_Module(nn.Module):

    def __init__(self, tovae, olf_config: dict, matprop_mpp_dict: dict):
        """
        Inputs for the gradient descent setup module

        Args:
            tovae: loaded instance of TO-VAE
            olf_config (dict): configuration dictionary for objective loss function
            matprop_mpp_dict (dict): dictionary that looks up the MPP that predicts each material property
        """

        super(GradOpt_Optimization_Loss_Module, self).__init__()

        self.olf_config = olf_config
        

        self.obj_fn = [Objective_Function(key, config_dict['objective variables'][key]) for key in config_dict['objective variables'].keys()]

        self.constraints = [MatProp_Constraints(key, value) for key, value in self.olf_config['constraints'].items()]

        self.matprops = []
        self.mpv_dict = {}

        idx = 0
        # loop through the configuration dictionary
        for key in olf_config.keys():
            sub = olf_config[key]
            for skey in sub.keys():
                # add to list of the material properties
                self.matprops.append(skey)

                #
                self.mpv_dict[skey] = idx
                idx += 1


        self.mpp_dict = matprop_mpp_dict

        self.mpps = nn.ModuleList()

        self.mpp_idx = []

        
        for mp in self.matprops:
            mpp_id = self.mpp_dict[mp][0] # pulls the name of the MPP based on the 
            mpp = getattr(tovae, mpp_id) # pulls the MPP module from the TOVAE
            self.mpps.append(mpp)        # adds to ModuleList
            self.mpp_idx.append(self.mpp_dict[mp][1]) # takes the index of the material property in the output of the MPP, e.g., if the MPP predicts KH_11, KH_22, and KH_33, then KH_22 is at index [1]

    def get_mpv_pred(self, latent_vec):
        """
        Produces material property prediction vector of selected material properties, based on the property name
        and its index in the output from its MPP. E.g., if the MPP predicts KH_11, KH_22, and KH_33, then KH_22 is at index [1]

        Args:
            latent_vec (torch.Tensor): latent vector as input

        Returns:
            torch.Tensor: tensor of the predicted material properties
        """

        preds = []
        for mpp, idx in zip(self.mpps, self.mpp_idx):
            pred = mpp(latent_vec.to(device))
            pred = list(pred.detach().cpu().numpy()[0,:])[idx]

            preds.append(pred)

        preds = torch.from_numpy(np.asarray(preds)).unsqueeze(0).to(device)

        return preds



    def forward(self, latent_vec):
        """
        Calls the objective function and constraints, adds them together, returns the total loss value

        Args:
            latent_vec (torch.Tensor): latent vector

        Returns:
            float: value of the loss function
        """

        self.mpv_pred = self.get_mpv_pred(latent_vec)

        loss_val = 0

        for i, obj_var in enumerate(self.olf_config['objective variables'].keys()):

            obj_var_idx = self.mpv_dict[obj_var]

            obj_var_pred = self.mpv_pred[obj_var_idx, :]

            obj_var_loss = self.obj_fn[i].calculate_objfn_value(obj_var_pred)

            loss_val += obj_var_loss

        for j, constr in enumerate(self.olf_config['constraints'].keys()):
            constr_idx = self.mpv_dict[constr]

            constr_pred = self.mpv_pred[:, constr_idx] # !!! 2x check on indexing [0,:] or [:,0]?

            constr_loss = self.constraints[j].calculate_loss_terms(constr_pred)

            loss_val += constr_loss

        return constr_loss

In [25]:
"""
matprop groups corresponding to which mlp[x]

mpp    group
0      ['volFrac',], 
1      ['CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',],
2      ['CH_12 scaled', 'CH_13 scaled','CH_23 scaled',],
3      ['EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',],
4      ['GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled',],
5      ['vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',],
6      ['KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled',],
7      ['kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']

"""


mpp_lookup = {
'mpp0': ['volFrac',], 
'mpp1': ['CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',],
'mpp2': ['CH_12 scaled', 'CH_13 scaled','CH_23 scaled',],
'mpp3': ['EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',],
'mpp4': ['GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled',],
'mpp5': ['vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',],
'mpp6': ['KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled',],
'mpp7': ['kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']
}

In [26]:
# For creating dict below again if necessary
# reverse_lookup = {}
# for key, value in mpp_lookup.items():
#     idx = 0
#     for par in value:
#         reverse_lookup[par] = (key, idx)
#         idx +=1

In [26]:
# This dictionary lists all material properties and identifies which MPP predicts them and which element in its output vector is the 

matprop_mpp_lookup = {'volFrac': ('mpp0', 0),
                 'CH_11 scaled': ('mpp1', 0),
                 'CH_22 scaled': ('mpp1', 1),
                 'CH_33 scaled': ('mpp1', 2),
                 'CH_44 scaled': ('mpp1', 3),
                 'CH_55 scaled': ('mpp1', 4),
                 'CH_66 scaled': ('mpp1', 5),
                 'CH_12 scaled': ('mpp2', 0),
                 'CH_13 scaled': ('mpp2', 1),
                 'CH_23 scaled': ('mpp2', 2),
                 'EH_11 scaled': ('mpp3', 0),
                 'EH_22 scaled': ('mpp3', 1),
                 'EH_33 scaled': ('mpp3', 2),
                 'GH_23 scaled': ('mpp4', 0),
                 'GH_13 scaled': ('mpp4', 1),
                 'GH_12 scaled': ('mpp4', 2),
                 'vH_12 scaled': ('mpp5', 0),
                 'vH_13 scaled': ('mpp5', 1),
                 'vH_23 scaled': ('mpp5', 2),
                 'vH_21 scaled': ('mpp5', 3),
                 'vH_31 scaled': ('mpp5', 4),
                 'vH_32 scaled': ('mpp5', 5),
                 'KH_11 scaled': ('mpp6', 0),
                 'KH_22 scaled': ('mpp6', 1),
                 'KH_33 scaled': ('mpp6', 2),
                 'kappaH_11 scaled': ('mpp7', 0),
                 'kappaH_22 scaled': ('mpp7', 1),
                 'kappaH_33 scaled': ('mpp7', 2)}

In [ ]:
output_dict = {}

if dynamic_topopt:
    dyn_val_dict = {}

# If desired to time each run, set to True
mark_time = True

for num in range(num_runs):
    run = num
    # Define a dictionary for the run
    rundic = {}

    """
    Select a random sample from the dataset as the starting point - 
    will be embedded in the latent space, producing the latent vector 
    that is the starting point for gradient descent optimization.

    The part number column value corresponds to the database
    """

    starting_pn = sdb.sample(n=1)['full PN'].values[0]
    
    rundic['seed PN'] = starting_pn
    
    # Set the filepath to the selected voxel array and then embed using get_latent_vec function
    arraypath = os.path.join(voxel_dir, f'{starting_pn}.npz')

    # Produce latent vector using the trained TO-VAE via get_latent_vec function
    lv_orig = get_latent_vec(tovae, arraypath)

    # store Latent Vector in the run dictionary for later reference
    rundic['starting LV'] = lv_orig.cpu().detach().numpy()
    
    # we call the starting latent vector the "guess"
    # this line creates a copy of the vector that will be integrated into the gradient descent process, i.e., attached to the computation graph
    lv_guess = lv_orig.clone().detach().requires_grad_(True)

    # Set the optimizer to target the guess latent vector
    optimizer = torch.optim.Adam([lv_guess], lr=0.01) 
    n_iter = 1
    
    # if desired, time the duration, otherwise, comment out this line
    if mark_time:
        start = time.time()
    else:
        pass

    # instantiate the loss function module:
    if optimiz_loss:
        gradopt_module = GradOpt_Optimization_Loss_Module(tovae, config_dict, matprop_mpp_lookup) # <><><>< Don't forget to change the name of reverse_lookup
        matprops = gradopt_module.matprops
    else:
        gradopt_module = GradOpt_MAE_Loss_module(tovae, config_dict, matprop_mpp_lookup)
        matprops = gradopt_module.matprops

    # iteration loop
    for iteration in range(num_iterations):
        optimizer.zero_grad()  # Clear previous gradients
       

        if optimiz_loss:
            # mpv_pred = gradopt_module.get_mpv_pred(lv_guess)   # LEFT OFF HERE... GOING THROUGH THE LOOP...
            loss = gradopt_module(lv_guess)
            loss.backward()
            optimizer.step()

        else:
            # Even if using Mean Absolute Error loss to target a specific MPV, the gradopt_module handles producing the predicted material property vector
            mpv_pred = gradopt_module.get_mpv_pred(lv_guess)
            loss = nn.L1Loss()(tgt_mpv, mpv_pred)
            loss.backward()
            optimizer.step

        n_iter += 1
    
    if mark_time:
        end = time.time()
        timetime = end - start
        print(f"iters time elapsed: {timetime}")
    else:
        pass
    
    # store final latent  vec
    rundic['final lv'] = lv_guess.detach().cpu().numpy()

    # get final mpv pred, detach and store
    mpv_pred_final = gradopt_module.get_mpv_pred(lv_guess).detach().cpu().numpy()
    mpv_pred_final = np.squeeze(mpv_pred_final, axis=0)

    preds_entry = []
    preds_lookup = {}

    for i, mp in enumerate(matprops):

        preds_entry.append(mpv_pred_final[i])
        if mp == 'volFrac':
            vf_pred = mpv_pred_final[i]
        elif mp == 'CH_11 scaled':
            c11_scaled_pred = mpv_pred_final[i]
            mp_name = mp.split(' ')[0]
            pred_unscaled = unscale(sdb, mp_name, mpv_pred_final[i])
            preds_entry.append(pred_unscaled)
        else:
            mp_name = mp.split(' ')[0]
            pred_unscaled = unscale(sdb, mp_name, mpv_pred_final[i])
            preds_entry.append(pred_unscaled)

    predsdf.loc[len(predsdf)] = preds_entry

    rundic['final predicted MPV'] = {prop: value for prop, value in zip(list(predsdf.columns), preds_entry)}

    
    lv_guess = torch.cat((lv_guess, tgt_vf_torch), dim=1)


    # calculate predicted plateau stress and energy absorbed using the final predicted CH_11 -- NOTE THAT THIS CH_11 MAY BE EXTREMELY INACCURATE AND CAN ONLY BE ACCURATE THROUGH FEA VALIDATION
    if dynamic_topopt:
        sig_pl_scaled = calculate_dynamic_property('plateau stress', c11_scaled_pred)
        sig_pl_unscaled = unscale(dyndb, 'plateau_stress_g', sig_pl_scaled[1])

        w_scaled = calculate_dynamic_property('energy absorbed', c11_scaled_pred)
        w_unscaled = unscale(dyndb, 'energy_absorbed_g', w_scaled[1])

        dyn_val_dict[f'run {run}'] = {'CH_11 scaled - TOVAE prediction': c11_scaled_pred,
                                      'plateau stress':{'scaled': sig_pl_scaled,
                                                        'unscaled': sig_pl_unscaled},
                                      'energy absorbed':{'scaled': w_scaled,
                                                        'unscaled': w_unscaled}}
        
    else:
        pass

        
    
    # store decoded 
    decoded = decoder(lv_guess).detach().cpu().numpy()
    decoded = np.squeeze(decoded, axis=(0,1))
    rundic['decoded array'] = decoded
    
    # store binary array, targeted to predicted volume fraction threshold
    binary_array = target_binarray_threshold(decoded, vf_pred)[0]
    rundic['binary array'] = binary_array
    
    # save binary array as .mat file for FEA homogenization in MATLAB
    matname = f'result{run}_{run_suffix}.mat'
    matpath = os.path.join(matdir, matname)

    matfile_dict = {'arr_0': binary_array}

    savemat(matpath, matfile_dict)

    # save plot of binary array
    binary_plotpath = os.path.join(plotdir, f'{matname[:-4]}_binary')
    Plot_Array(binary_array, binary=True, marker_size=5, symbol='square', show=False, export_png=True, plotpath=binary_plotpath)

    # save plot of continuous array
    continuous_plotpath = os.path.join(plotdir, f'{matname[:-4]}_continuous')
    Plot_Array(decoded, binary=False, marker_size=5, symbol='square', show=False, export_png=True, scale_markers=True, plotpath=continuous_plotpath)
    
    
    # add run to full dictionary
    output_dict[f'run {run}'] = rundic
    

In [29]:
# Save dictionary of arrays as desired


output_dict_pkl_name = f'{batch_folder_name}_run_dict.pkl'
pklpath = os.path.join(batch_path, output_dict_pkl_name)
save_dict_to_pickle(pklpath, output_dict)


In [30]:
# Save dynamic results dict

output_dyndict_pkl_name = f'{batch_folder_name}_dynamic_properties_run_dict.pkl'
pklpath = os.path.join(batch_path, output_dyndict_pkl_name)
save_dict_to_pickle(pklpath, dyn_val_dict)

In [31]:
# Save results dataframe to csv file

df_name = f'{batch_folder_name}_results_table.csv'
dfpath = os.path.join(batch_path, df_name)
predsdf.to_csv(dfpath)